In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
from decimal import Decimal
import pandas as pd
import os, sys
from time import sleep
import urllib.request
from random import randint
from warnings import warn
from time import time, ctime
from IPython.core.display import clear_output
import numpy as np

In [2]:
mpgs = pd.read_csv('data/mpg.csv', usecols = ['make','model','year','city08','comb08','highway08','trany','drive'])

In [3]:
pics = 1 # 1 for yes
distance = 35 # miles from zip
zipcode = 98402
minp = 2500 # min price
maxp = 16000 # max price
miny = 2005 # min auto year
maxo = 150000 # max odometer reading

In [4]:
pics = "hasPic="+str(pics)+"&"
distance = "search_distance="+str(distance)+"&"
zipcode = "postal="+str(zipcode)+"&"
minp = "min_price="+str(minp)+"&"
maxp = "max_price="+str(maxp)+"&"
miny = "min_auto_year="+str(miny)+"&"
maxo = "max_auto_miles="+str(maxo)+"&"

conds = "condition=10&condition=20&condition=30&condition=40&" # exclude fair condition and below
fuel = "auto_fuel_type=1&auto_fuel_type=3&auto_fuel_type=4&" # gas, hybrid, and electric
title = "auto_title_status=1&" # clean title
tran = "auto_transmission=2&" # automatic transmission
body = "auto_bodytype=3&auto_bodytype=4&auto_bodytype=8&auto_bodytype=10&auto_bodytype=11&" # sedan, coupe, wagon, suv, hatchback

search_str = "https://seattle.craigslist.org/search/sso?"+pics+distance+zipcode+minp+maxp+miny+maxo+conds+fuel+title+tran+body
print(search_str)

https://seattle.craigslist.org/search/sso?hasPic=1&search_distance=35&postal=98402&min_price=2500&max_price=16000&min_auto_year=2005&max_auto_miles=150000&condition=10&condition=20&condition=30&condition=40&auto_fuel_type=1&auto_fuel_type=3&auto_fuel_type=4&auto_title_status=1&auto_transmission=2&auto_bodytype=3&auto_bodytype=4&auto_bodytype=8&auto_bodytype=10&auto_bodytype=11&


In [5]:
search_response = get(search_str)
search_soup = BeautifulSoup(search_response.text, 'html.parser')
results_n = search_soup.find('div', class_= 'search-legend')
N = int(results_n.find('span', class_='totalcount').text)
print("Found "+str(N)+" posts to scrape!")

Found 304 posts to scrape!


In [6]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Scraper progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [7]:
d = pd.DataFrame(columns=['url',
                          'post_name',
                          'year',
                          'make_mod',
                          'make',
                          'mod',
                          'price',
                          'location',
                          'vin',
                          'condition',
                          'drive',
                          'fuel',
                          'odometer',
                          'paint',
                          'title',
                          'transmission',
                          'type',
                          'size',
                          'cylinders',
                          'city_mpg',
                          'hwy_mpg',
                          'comb_mpg',
                          'description',
                          'imgurl'])

pages = np.arange(0, N+1, 120)
iterations = 0
u = 0

for page in pages:
    
    sleep(randint(1,5))
    search = search_str + "s=" + str(page)
    search_response = get(search)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    posts = search_soup.find_all('li', class_= 'result-row')
    
    for post in range(0,len(posts)):
        
        sleep(randint(1,5))
        url = posts[post].find('a', class_='result-title hdrlnk')['href']
        response = get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if soup.find(id="titletextonly") is not None:
            
            d.loc[u,'url'] = url
            d.loc[u,'post_name'] = soup.find(id="titletextonly").text.strip()
            d.loc[u,'year'] = int(soup.find(class_='attrgroup').b.text[:4].strip())
            d.loc[u,'make_mod'] = soup.find(class_='attrgroup').b.text[5:].strip().lower()
            d.loc[u,'make'] = d.loc[u,'make_mod'].split(' ')[0].lower()
            if len(d.loc[u,'make_mod'].split(' ')) > 1:
                d.loc[u,'mod'] = d.loc[u,'make_mod'].split(' ')[1].lower()
            money = soup.find(class_="price").text.strip()
            d.loc[u,'price'] = Decimal(re.sub(r'[^\d.]', '', money))
            d.loc[u, 'location'] = re.sub('\W+',' ', soup.small.text.strip())

            for span_tag in soup.find_all(class_='attrgroup')[1].find_all('span'):
                if span_tag.text.split(':')[0]=='VIN':
                    d.loc[u, 'vin'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='condition':
                    d.loc[u, 'condition'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='drive':
                    d.loc[u, 'drive'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='fuel':
                    d.loc[u, 'fuel'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='odometer':
                    d.loc[u, 'odometer'] = Decimal(span_tag.text.split(':')[1].strip())
                elif span_tag.text.split(':')[0]=='paint color':
                    d.loc[u, 'paint'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='title status':
                    d.loc[u, 'title'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='transmission':
                    d.loc[u, 'transmission'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='type':
                    d.loc[u, 'type'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='size':
                    d.loc[u, 'size'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='cylinders':
                    d.loc[u, 'cylinders'] = span_tag.text.split(':')[1].strip()

            if not mpgs[(mpgs.year == d.loc[u,'year']) &
                    (mpgs.make.str.lower() == d.loc[u,'make']) &
                    (mpgs.model.str.split().str.get(0).str.lower() == d.loc[u,'mod']) &
                    (mpgs.trany.str.split().str.get(0).str.lower() == d.loc[u,'transmission'])
                        ].empty:
                d_mpg = mpgs[(mpgs.year == d.loc[u,'year']) &
                        (mpgs.make.str.lower() == d.loc[u,'make']) &
                        (mpgs.model.str.split().str.get(0).str.lower() == d.loc[u,'mod']) &
                        (mpgs.trany.str.split().str.get(0).str.lower() == d.loc[u,'transmission'])
                            ].iloc[0]
                d.loc[u, 'city_mpg'] = int(d_mpg.city08)
                d.loc[u, 'hwy_mpg'] = int(d_mpg.highway08)
                d.loc[u, 'comb_mpg'] = int(d_mpg.comb08)

            d.loc[u, 'description'] = soup.find(id='postingbody').text[30:]
            d.loc[u, 'imgurl'] = soup.find(class_="slide first visible").img['src']
        
        update_progress(u / N)
        u += 1
       
    iterations += 1
    update_progress(1) 

d.description = d.description.str.replace('\n', '\\')

print("\n")
print("Done!")  

Scraper progress: [####################] 100.0%


Done!


In [8]:
d

,url,post_name,year,make_mod,make,mod,price,location,vin,condition,...,title,transmission,type,size,cylinders,city_mpg,hwy_mpg,comb_mpg,description,imgurl
0,https://seattle.craigslist.org/skc/cto/d/maple...,2012 Chevy Cruze LT,2012,chevy cruze,chevy,cruze,6500,Maple Valley,1G1PF5SC2C7107380,excellent,...,clean,automatic,sedan,compact,4 cylinders,NaN,NaN,NaN,"We bought this car used in 2013, and we are se...",https://images.craigslist.org/01212_qH6lCQZvay...
1,https://seattle.craigslist.org/see/cto/d/rento...,Chevrolet Equinox,2005,chevrolet equinox,chevrolet,equinox,4500,Renton,NaN,good,...,clean,automatic,SUV,mid-size,6 cylinders,17,23,19,2005 Chevrolet Equinox - Price marked down to ...,https://images.craigslist.org/00O0O_fdldeBbs3C...
2,https://seattle.craigslist.org/oly/cto/d/olymp...,2017 Nissan Rogue S FWD,2017,nissan rogue,nissan,rogue,15950,Olympia,JN8AT2MT5HW381577,excellent,...,clean,automatic,SUV,mid-size,4 cylinders,26,33,29,"Exterior/Body\Rear Defrost, Backup Camera, Rea...",https://images.craigslist.org/00J0J_4FKtjSLFAE...
3,https://seattle.craigslist.org/see/cto/d/belle...,"2011 Mercedes Benz C300 Sport Sedan, Clean Tit...",2011,mercedes-benz c-class c300,mercedes-benz,c-class,11200,Bellevue,NaN,excellent,...,clean,automatic,sedan,compact,6 cylinders,NaN,NaN,NaN,Year: 2011\Make: Mercedes\Model: c300\Millage:...,https://images.craigslist.org/00R0R_fEsf2o5FU7...
4,https://seattle.craigslist.org/skc/cto/d/seatt...,"2008 2wd Escape, 98800 miles, new transmissio...",2008,ford escape,ford,escape,7000,Des Moines,1fmcu02z58ka91081,excellent,...,clean,automatic,SUV,mid-size,4 cylinders,20,26,22,Selling my 2008 Ford Escape. It has been a gre...,https://images.craigslist.org/00606_4qgHb15Hy8...
5,https://seattle.craigslist.org/tac/cto/d/lakew...,2013 SHO Taurus,2013,ford sho taurus,ford,sho,9000,At the shop,NaN,excellent,...,clean,automatic,sedan,full-size,6 cylinders,NaN,NaN,NaN,SHO MODEL! \The good- Pearl White. Suede inser...,https://images.craigslist.org/01313_9RKYRy0Ceb...
6,https://seattle.craigslist.org/skc/cto/d/seatt...,2009 Cadillac CTS 4 AWD,2009,cadillac cts 4,cadillac,cts,8500,Burien,1G6DG577X90135614,excellent,...,clean,automatic,sedan,NaN,6 cylinders,17,26,20,Ive had this car for several years and have lo...,https://images.craigslist.org/00505_kOVRHfi8qk...
7,https://seattle.craigslist.org/tac/cto/d/gig-h...,Volvo S60 T5 2012 4D,2012,volvo s60,volvo,s60,8700,Gig Harbor WA,YV1622FS6C2091254,excellent,...,clean,automatic,sedan,NaN,5 cylinders,18,26,21,This has been a great commuter car. Regular m...,https://images.craigslist.org/00N0N_l0Nx66ogB1...
8,https://seattle.craigslist.org/see/cto/d/seatt...,Honda Accord 2008 low miles,2008,honda accord ex-l sedan 4 dr,honda,accord,8500,google map,NaN,excellent,...,clean,automatic,sedan,NaN,4 cylinders,21,30,24,"One family owner, low miles, my Mother’s car. ...",https://images.craigslist.org/00l0l_8EF7Cyi65l...
9,https://seattle.craigslist.org/tac/cto/d/gig-h...,2006 Nissan Xterra S,2006,nissan xterra,nissan,xterra,3900,Gig Harbor,NaN,good,...,clean,automatic,SUV,NaN,NaN,14,20,16,2006 NISSAN XTERRA S\Vehicle Info\Condition: U...,https://images.craigslist.org/00T0T_f4dEOIFmc0...


In [9]:
d.to_csv(r'data/cars.csv', index=True, header=True)